# Clustering

In this example we will do some simple cell clustering based on multiband imagery in the region around Elkton, Virginia.

## Setup 

First some setup. We import all the necessary libraries and create our `SparkSession`.

In [1]:
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pathlib import Path

spark = SparkSession.builder. \
    master("local[*]"). \
    appName("RasterFrames"). \
    config("spark.ui.enabled", "false"). \
    getOrCreate(). \
    withRasterFrames()

## Loading Data

The first step is to load multiple bands of imagery and construct a single RasterFrame from them. To do this we:

1. Identify the GeoTIFF filename.
2. Read the TIFF raster
3. Create the names of our columns
4. Define a function for reading in a RasterFrame based on a band number
5. Use the RasterFrames spatialJoin function to create a new RasterFrame with a column for each band

In [2]:
resource_dir = Path('./samples').resolve()
# Utility for reading imagery from our test data set
filenamePattern = "L8-B{}-Elkton-VA.tiff"
bandNumbers = range(1, 5)
bandColNames = ['band_{}'.format(n) for n in bandNumbers]

def readTiff(band):
    return spark.read.geotiff(resource_dir.joinpath(filenamePattern.format(band)).as_uri())

In [3]:
from functools import reduce
joinedRF = reduce(lambda rf1, rf2: rf1.asRF().spatialJoin(rf2.drop('bounds').drop('metadata')),
                  [readTiff(b).withColumnRenamed('tile', 'band_{}'.format(b)) for b in bandNumbers])

We should see a `spatial_key` column, `bounds`, `metadata`, and 4 columns of tiles, representing our four bands.

In [4]:
joinedRF.printSchema()

root
 |-- spatial_key: struct (nullable = false)
 |    |-- col: integer (nullable = false)
 |    |-- row: integer (nullable = false)
 |-- bounds: polygon (nullable = true)
 |-- metadata: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = false)
 |-- band_1: rf_tile (nullable = false)
 |-- band_2: rf_tile (nullable = false)
 |-- band_3: rf_tile (nullable = false)
 |-- band_4: rf_tile (nullable = false)



## ML Pipeline 

PySparkML requires that each observation be in its own row, and those
observations be packed into a single `Vector`. The first step is to
"explode" the tiles into a single row per cell/pixel.

In [5]:
exploder = TileExploder()

To "vectorize" the the band columns, as required by SparkML, we use the SparkML 
`VectorAssembler`. We then configure our algorithm, create the transformation pipeline,
and train our model. (Note: the selected value of *K* below is arbitrary.) 

In [6]:
assembler = VectorAssembler() \
    .setInputCols(bandColNames) \
    .setOutputCol("features")

# Configure our clustering algorithm
k = 5
kmeans = KMeans().setK(k)

# Combine the three stages
pipeline = Pipeline().setStages([exploder, assembler, kmeans])

# Fit the model
model = pipeline.fit(joinedRF)

## Model Evaluation

Once trained, the model can be saved off for later use, or used immediately on the same
data we used to compute the model. First, we run the data through the model to compute 
cluster IDs for each cell.

In [7]:
clustered = model.transform(joinedRF)
clustered.show(3)

+-----------+--------------------+--------------------+------------+---------+------+------+------+------+--------------------+----------+
|spatial_key|              bounds|            metadata|column_index|row_index|band_1|band_2|band_3|band_4|            features|prediction|
+-----------+--------------------+--------------------+------------+---------+------+------+------+------+--------------------+----------+
|      [0,0]|POLYGON ((703986....|Map(AREA_OR_POINT...|           0|        0|9470.0|8491.0|7805.0|6697.0|[9470.0,8491.0,78...|         0|
|      [0,0]|POLYGON ((703986....|Map(AREA_OR_POINT...|           1|        0|9566.0|8607.0|8046.0|6898.0|[9566.0,8607.0,80...|         0|
|      [0,0]|POLYGON ((703986....|Map(AREA_OR_POINT...|           2|        0|9703.0|8808.0|8377.0|7222.0|[9703.0,8808.0,83...|         2|
+-----------+--------------------+--------------------+------------+---------+------+------+------+------+--------------------+----------+
only showing top 3 rows



In [8]:
clusterResults = list(filter(lambda x: str(x).startswith('KMeans'), model.stages))[0]

Compute sum of squared distances of points to their nearest center:

In [9]:
metric = clusterResults.computeCost(clustered)
print("Within set sum of squared errors: %s" % metric)

Within set sum of squared errors: 10442521284.334034


## Visualizing Results

The predictions are in a DataFrame with each row representing a separate pixel. 
To assemble a raster to visualize the cluster assignments, we have to go through a
multi-stage process to get the data back in tile form, and from there to combined
raster form.

First, we get the DataFrame back into RasterFrame form:

In [10]:
tlm = joinedRF.tileLayerMetadata()
layout = tlm['layoutDefinition']['tileLayout']

retiled = clustered.groupBy('spatial_key').agg(
    assembleTile('column_index', 'row_index', 'prediction',
        layout['tileCols'], layout['tileRows'], 'int8')
)

rf = retiled.asRF('spatial_key', tlm)
rf.printSchema()
rf.show(truncate=False)

root
 |-- spatial_key: struct (nullable = false)
 |    |-- col: integer (nullable = false)
 |    |-- row: integer (nullable = false)
 |-- prediction: rf_tile (nullable = true)

+-----------+------------------------------------------------+
|spatial_key|prediction                                      |
+-----------+------------------------------------------------+
|[0,0]      |ByteConstantNoDataArrayTile([B@3275ca1b,186,169)|
+-----------+------------------------------------------------+



To render our visualization, we convert to a raster first, which gives us a 1-D array with all the cell values. We then reshape the raster into a 2d array and use
matplotlib colormap to assign each cluster a different color and then graph the resulting image.

In [11]:
# takes a while
import numpy as np
raster = np.asarray(list(rf.toIntRaster('prediction', 186, 169)))

pixels = np.reshape(raster, (169, 186))

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(16,16))
fig.add_subplot(1, 2, 1)
plt.set_cmap('Accent')
plt.imshow(pixels)
fig.add_subplot(1, 2, 2)
pixels2 = mpl.pyplot.imread("pics/L8-RGB-VA.png")
plt.imshow(pixels2)

The colors in the first picture are just the colormap's representation of the clusters assigned by the model, which are ints between 0 and 4, inclusive. The picture on the right is an RGB image of the scene. As we can see, blue clusters appear to be small centers of intense urban activity. Brown clusters are urban activity, green corresponds closely to roads and pavement, gray appears to be farmland, and orange matches closely with forested regions.